# 1. Simple mode (no symmetry)

## 1.1 Plot bands from the DFT code

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import scipy

bands_pw = np.loadtxt("./data/diamond.bands_pw.dat.gnu")

bohr_ang = scipy.constants.physical_constants['Bohr radius'][0] / 1e-10
alatt = 6.1* bohr_ang
bands_pw[:,0]*= 2*np.pi/alatt

plt.scatter(bands_pw[:, 0], bands_pw[:, 1], s=1, c="black", label="PW bands")
plt.show()



## 2.2 Load Wannier90 files and wannierize them


In [ ]:
from wannierberri.w90files import Wannier90data

w90data = Wannier90data().from_w90_files("./data/w90files/diamond", readfiles=["win", "mmn", "amn", "eig", "unk"])

froz_max=22

w90data.select_bands(win_min=-np.inf, win_max = froz_max)

w90data.wannierise( init = "amn",
                froz_min=-np.inf,
                froz_max=froz_max,
                print_progress_every=10,
                num_iter=41,
                conv_tol=1e-6,
                mix_ratio_z=1.0,
                sitesym=False,
                )


## 1.3 Plot the Wannier functions

In [ ]:
w90data.plotWF(path="./diamond-bond")
%ls
%pwd

## 1.4 Create a System() object

This object may be created in many ways (from the .chk file by wannier90, or from FPLO, ASE, or a tight-binding model, even a k.p model)

In [ ]:
from wannierberri.system import System_w90
system = System_w90(w90data=w90data)


## 1.5 calculate bands along the path and plot them

In [ ]:
from wannierberri.grid import Path
from wannierberri import evaluate_k_path

path = Path(system=system,
            nodes = [[1/2,1/2,1/2], [0,0,0], [1/2,-1/2,0], [3/8, -3/8, 0], [0,0,0]],
            labels = ["L", "$\Gamma$", "X", "K", "$\Gamma$"],
            length=500)

bands_wb = evaluate_k_path(system=system, path=path)

plt.scatter(bands_pw[:,0], bands_pw[:,1], c="black", s=5)



bands_wb.plot_path_fat(path,
                quantity=None,
                # save_file="Fe_bands.pdf",
                Eshift=0,
                Emin=-10, Emax=50,
                iband=None,
                mode="fatband",
                fatfactor=20,
                cut_k=False,
                linecolor="red",
                close_fig=False,
                show_fig=False,
                label=f"WB"
                )

# plt.ylim(-10, 40)
plt.hlines(froz_max, 0, bands_pw[-1,0], linestyles="dashed", label="frozen window", color="black")
plt.legend()
plt.show()

## 1.6 Change the projections

### load the bandstructure directly from DFT code outputs

In [ ]:
from wannierberri.symmetry.projections import Projection, ProjectionsSet
from wannierberri.w90files import AMN
from irrep.bandstructure import BandStructure

bandstructure = BandStructure(code="espresso", prefix="./data/di")
sg = bandstructure.spacegroup
# sg.show()

positions_atoms = [[1/8,1/8,1/8], [-1/8,-1/8,-1/8]] 
proj_s = Projection(position_num=positions_atoms, orbital="s", spacegroup=sg)
proj_p = Projection(position_num=positions_atoms, orbital="p", spacegroup=sg)

projections = ProjectionsSet([proj_s, proj_p])

# This requires wannierberri ==1.4.2
# proj_sp3 = Projection(position_num=positions_atoms, orbital="sp3", 
#                       basis_list=[np.eye(3), -np.eye(3)],
#                       spacegroup=sg)
# projections = ProjectionsSet([proj_sp3])



amn_new = AMN.from_bandstructure(bandstructure=bandstructure, projections=projections)

w90data2 = Wannier90data().from_w90_files("./data/w90files/diamond", readfiles=["win", "mmn", "eig"])
w90data2.set_file("amn", amn_new)


froz_max=35
# w90data.select_bands(win_min=-np.inf, win_max = 100)

w90data2.wannierise(
                init = "amn",
                froz_min=-np.inf,
                froz_max=froz_max,
                print_progress_every=10,
                num_iter=101,
                conv_tol=1e-6,
                mix_ratio_z=1.0,
                sitesym=False,
                )


In [ ]:
system2 = System_w90(w90data=w90data2)
bands_wb = evaluate_k_path(system=system2, path=path)

plt.scatter(bands_pw[:,0], bands_pw[:,1], c="black", s=5)



bands_wb.plot_path_fat(path,
                quantity=None,
                # save_file="Fe_bands.pdf",
                Eshift=0,
                Emin=-10, Emax=50,
                iband=None,
                mode="fatband",
                fatfactor=20,
                cut_k=False,
                linecolor="red",
                close_fig=False,
                show_fig=False,
                label=f"WB"
                )

# plt.ylim(-10, 40)
plt.hlines(froz_max, 0, bands_pw[-1,0], linestyles="dashed", label="frozen window", color="black")
plt.legend()
plt.show()

# 2. Symmetry-adapted Wannier Functions

In [ ]:
from wannierberri.symmetry.sawf import SymmetrizerSAWF

positions_bonds = [[ 0,0,0], [0,0,1/2], [0,1/2,0], [1/2,0,0] ]
proj_s_bond = Projection(position_num=positions_bonds,  orbital="s", spacegroup=sg)
proj_p_bond = Projection(position_num = positions_bonds, orbital='pz', 
                         zaxis =  np.array([1,1,1])@sg.lattice, 
                         rotate_basis=True,
                         spacegroup=sg)

proj_sp3 = Projection(position_num=positions_atoms, orbital="sp3", spacegroup=sg, rotate_basis=True)

projections = ProjectionsSet([proj_s_bond])
projections = ProjectionsSet([proj_s_bond, proj_p_bond])
projections = ProjectionsSet([proj_sp3])
projections = ProjectionsSet([proj_s, proj_p])


amn_new = AMN.from_bandstructure(bandstructure=bandstructure, projections=projections)

w90data3 = Wannier90data().from_w90_files("./data/w90files/diamond", readfiles=["win", "mmn", "eig", "unk"])
w90data3.set_file("amn", amn_new)



symmetrizer = SymmetrizerSAWF().from_irrep(bandstructure=bandstructure)
symmetrizer.set_D_wann_from_projections(projections=projections)


w90data3.set_symmetrizer(symmetrizer)
froz_max=30
w90data3.wannierise(
                init = "amn",
                froz_min=-np.inf,
                froz_max=froz_max,
                print_progress_every=10,
                num_iter=101,
                conv_tol=1e-6,
                mix_ratio_z=1.0,
                sitesym=True,
                )


In [ ]:
system3 = System_w90(w90data=w90data3, symmetrize=False)
bands_wb = evaluate_k_path(system=system3, path=path)

plt.scatter(bands_pw[:,0], bands_pw[:,1], c="black", s=5)



bands_wb.plot_path_fat(path,
                quantity=None,
                # save_file="Fe_bands.pdf",
                Eshift=0,
                Emin=-10, Emax=50,
                iband=None,
                mode="fatband",
                fatfactor=20,
                cut_k=False,
                linecolor="red",
                close_fig=False,
                show_fig=False,
                label=f"WB"
                )

# plt.ylim(-10, 40)
plt.hlines(froz_max, 0, bands_pw[-1,0], linestyles="dashed", label="frozen window", color="black")
plt.legend()
plt.show()